### Define Distribution

In [ ]:
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd

dict = [{"Treatment Num": 1, "Treatment Name": "Triage", "Time Distribution":"expo", "Parameter":[7]}, 
        {"Treatment Num": 2, "Treatment Name": "Registration", "Time Distribution":"expo", "Parameter":[5.5]}, 
        {"Treatment Num": 3, "Treatment Name": "Evaluation", "Time Distribution":"normal", "Parameter":[14,6]}, 
        {"Treatment Num": 4, "Treatment Name": "Laboratory", "Time Distribution":"normal", "Parameter":[35,15]}, 
        {"Treatment Num": 5, "Treatment Name": "X-ray", "Time Distribution":"expo", "Parameter":[12]}, 
        {"Treatment Num": 6, "Treatment Name": "Consultation", "Time Distribution":"normal", "Parameter":[15,18]}, 
        {"Treatment Num": 7, "Treatment Name": "CT scan", "Time Distribution":"normal", "Parameter":[29,14]}, 
        {"Treatment Num": 8, "Treatment Name": "Discharge", "Time Distribution":"constant", "Parameter":[30]}, 
        {"Treatment Num": 9, "Treatment Name": "Admission", "Time Distribution":"expo", "Parameter":[3]}]

df = pd.DataFrame(dict)
df

### Generate Patient Acuity Level

In [82]:
import random

def generate_acuity_level():
    # Define the acuity level array
    acuitylevel = [1, 2, 3, 4, 5]

    # Randomly select a number from the acuitylevel list
    random_acuity = random.choice(acuitylevel)
    return random_acuity

### Generate Patient Treatment Pattern

In [83]:
import random

def generate_treatment_pattern():
    patterns = {
        "pattern1": [1, 2, 3, 8],
        "pattern2": [1, 2, 3, 5, 6],
        "pattern3": [1, 2, 3, 4, 5, 8],
        "pattern4": [1, 2, 3, 6, 8],
        "pattern5": [1, 2, 3, 4, 5, 6, 8],
        "pattern6": [1, 2, 3, 4, 8],
        "pattern7": [1, 2, 3, 5, 4, 8],
        "pattern8": [1, 2, 3, 7, 8],
        "pattern9": [1, 2, 3, 7, 5, 6, 9],
        "pattern10": [1, 2, 3, 4, 9],
        "pattern11": [1, 2, 3, 5, 4, 9]
    }
    
    # Select a random pattern
    random_pattern_name = random.choice(list(patterns.keys()))
    random_pattern = patterns[random_pattern_name]

    return random_pattern

### Generate Patient Treatment Duration 

In [87]:
# def generate_expo(rate):
#     random = np.random.exponential(1 / rate)
#     return random

# def generate_gaussian(mean, std_dev):
#     return np.random.normal(mean, std_dev)

# Function to generate a random value from the exponential distribution
def generate_expo(rate):
    # Generate a value from the exponential distribution and round to the nearest integer
    random = np.random.exponential(1 / rate)
    return int(np.round(random))

# Function to generate a random value from the normal distribution
def generate_gaussian(mean, std_dev):
    # Generate a value from the normal distribution and round to the nearest integer
    random = np.random.normal(mean, std_dev)
    return int(np.round(random))

def generate_treatment_time(distribution, parameter):

    if distribution == "expo":
        rate = parameter[0]
        while True: #time can't be negative
            random = generate_expo(rate=rate)
            if random > 0:
                return random
    elif distribution == "normal":
        mean = parameter[0]
        std_dev = parameter[1]
        while True: #time can't be negative
            random = generate_gaussian(mean=mean, std_dev=std_dev)
            if random > 0:
                return random
    elif distribution == "constant":
        const = parameter[0]
        return const
    
def generate_treatment_totaltime_arr(df, pattern):
    
    arr = []
    for treatment_num in pattern:
        # Access row by Treatment Num
        row = df.loc[df['Treatment Num'] == treatment_num]
        
        if row.empty:
            print(f"Warning: Treatment Num {treatment_num} not found in DataFrame.")
            return  # Skip this treatment if not found
        
        # Ensure proper access to 'Time Distribution' and 'Parameter' values
        distribution = row["Time Distribution"].values[0]
        parameter = row["Parameter"].values[0]
       
        # Assuming `generate_treatment_time` is correctly implemented
        treatment_time = generate_treatment_time(distribution=distribution, parameter=parameter)
          
        arr.append(treatment_time)

    return arr

In [ ]:
# Function to plot the distribution
def plot_distribution(distribution, parameter, bins=100, iteration=10000):
    # Generate the data using the generate_treatment_time function
    data = [generate_treatment_time(distribution=distribution, parameter=parameter) for _ in range(iteration)]

    # Plot the histogram for the empirical data
    plt.hist(data, bins=bins, density=True, alpha=0.6, color='b', edgecolor='black')

    # Generate and plot the theoretical density function for comparison
    if distribution == 'expo':
        x = np.linspace(0, max(data), iteration)  # x-values for plotting the theoretical PDF
        rate = parameter[0]  # Rate parameter for exponential distribution
        y = rate * np.exp(-rate * x)  # Exponential PDF formula
    elif distribution == "normal":
        x = np.linspace(0, max(data), iteration)  # x-values for plotting the theoretical PDF
        mean = parameter[0]  # Mean of the normal distribution
        std_dev = parameter[1]  # Standard deviation of the normal distribution
        y = (1 / (std_dev * np.sqrt(2 * np.pi))) * np.exp(-0.5 * ((x - mean) / std_dev) ** 2)  # Normal PDF formula
    elif distribution == "constant":
        const = parameter[0]  # Constant distribution
        x = np.linspace(const-10, const+10, iteration)  # x-values for plotting the theoretical PDF
        y = np.ones_like(x)*(iteration/bins)

    # Plot the theoretical PDF
    plt.plot(x, y, 'r-', label='Theoretical PDF')

    # Add labels and title
    plt.xlabel('Value')
    plt.ylabel('Density')
    plt.title(f'{distribution.capitalize()} Distribution')
    plt.legend()
    plt.show()

# Function to test distributions using the given DataFrame
def test_distribution(df):
    for index, row in df.iterrows():
        treatment_num = row["Treatment Num"]
        treatment_name = row["Treatment Name"]
        time_distribution = row["Time Distribution"]
        parameter = row["Parameter"]

        # Print treatment information (optional)
        print(f"Treatment {treatment_num}: {treatment_name}, {time_distribution}, {parameter}")
        
        # Plot the distribution for each treatment
        plot_distribution(distribution=time_distribution, parameter=parameter)

test_distribution(df)

### Generate Patient Arrival Time

In [ ]:
def generate_poisson_arrivals(rate, max_time=60):
    arrival_times = []  # List to store valid arrival times
    arrival_time = 0

    # The rate parameter is given as patients per hour, so we convert it to rate per minute
    param = rate / 60  # Rate per minute

    # Generate inter-arrival times using the Poisson process
    while arrival_time <= max_time:
        # Generate the next inter-arrival time from the exponential distribution
        inter_arrival_time = np.random.exponential(1 / param)
        
        # Add the inter-arrival time to the cumulative arrival time
        arrival_time += inter_arrival_time
        
        # Only add arrival time to the list if it's within the max_time (and rounded to an integer)
        if arrival_time <= max_time:
            arrival_times.append(int(round(arrival_time)))  # Round to nearest whole minute
    
    return arrival_times


In [ ]:
import numpy as np
import matplotlib.pyplot as plt
from scipy.special import factorial

def plot_patient_arrivals(arrival_times):
    # Plot the patient arrival times
    plt.figure(figsize=(10, 6))
    plt.hist(arrival_times, bins=100, alpha=0.6, color='b', edgecolor='black')
    plt.title('Histogram of Patient Arrival Times (Poisson Process)')
    plt.xlabel('Time (minutes)')
    plt.ylabel('Probability Density')
    plt.grid(True)
    plt.show()
    
def plot_patient_arrivals(arrival_times):
    # Plot the patient arrival times
    plt.figure(figsize=(10, 6))
    plt.hist(arrival_times, bins=30, density=True, alpha=0.6, color='b', edgecolor='black')
    plt.title('Histogram of Patient Arrival Times (Poisson Process)')
    plt.xlabel('Time (minutes)')
    plt.ylabel('Probability Density')
    plt.grid(True)
    plt.show()

def plot_patient_arrivals_trial(arrival_counts, expected_rate):
    # Plot the patient arrival counts (for multiple trials)
    plt.figure(figsize=(10, 6))
    plt.hist(arrival_counts, bins=range(min(arrival_counts), max(arrival_counts) + 1), alpha=0.6, color='b', edgecolor='black', density=True)

    # Add the theoretical Poisson distribution for comparison
    x = np.arange(min(arrival_counts), max(arrival_counts) + 1)
    expected = (expected_rate ** x) * np.exp(-expected_rate) / factorial(x)
    plt.plot(x, expected, 'r-', label=f'Theoretical Poisson (λ={expected_rate})')

    plt.title('Histogram of Patient Arrival Counts (Poisson Process)')
    plt.xlabel('Number of Patients in an Hour')
    plt.ylabel('Probability Density')
    plt.legend()
    plt.grid(True)
    plt.show()

def test_poisson_arrivals():
    # Set the Poisson rate parameter (7 patients per hour)
    rate = 7
    num_trials = 1000  # Number of trials to run
    arrival_counts = []  # List to store the number of arrivals for each trial

    arrival_times = generate_poisson_arrivals(rate)
    # Print the arrival times
    print(f"Patient Arrival Times (minutes): {arrival_times}")
    # Plot the distribution of patient arrival times
    plot_patient_arrivals(arrival_times)

    # Run the simulation multiple times
    for _ in range(num_trials):
        arrival_times = generate_poisson_arrivals(rate)
        arrival_counts.append(len(arrival_times))  # Store the number of arrivals in this trial

    # Plot the distribution of patient arrival counts
    plot_patient_arrivals_trial(arrival_counts, expected_rate=rate)

test_poisson_arrivals()


### Generate Patient Data

In [94]:
total_hour = 10
rate = 7

'''     self.patient_id = patient_id                            
        self.arrival_time = arrival_time                        
        self.acuity_level = acuity_level                        
        self.treatment_plan_arr = treatment_plan_arr            
        self.treatment_totaltime_arr = treatment_totaltime_arr 
'''
columns = ['patient_id', 'arrival_time', 'acuity_level', 'treatment_plan_arr', 'treatment_totaltime_arr']
patients_df = pd.DataFrame(columns=columns)

patient_id_count = 1
for hour in range(total_hour):
    print("Hour: ", hour)
    patient_arrival_times =generate_poisson_arrivals(rate)
    print("Patient Arrival Time:", patient_arrival_times)

    for patient_minute in patient_arrival_times:
        patient_id = patient_id_count
        arrival_time = hour*60+patient_minute
        acuity_level = generate_acuity_level()
        treatment_plan_arr = generate_treatment_pattern()
        treatment_totaltime_arr = generate_treatment_totaltime_arr(df, treatment_plan_arr)

        new_row = pd.DataFrame([{
                'patient_id': patient_id,
                'arrival_time': arrival_time,
                'acuity_level': acuity_level,
                'treatment_plan_arr': treatment_plan_arr,
                'treatment_totaltime_arr': treatment_totaltime_arr,
                }])

        patients_df = pd.concat([patients_df, new_row], ignore_index=True)

        patient_id_count += 1

patients_df

Hour:  0
Patient Arrival Time: [14, 15, 19, 26, 31, 35, 43, 47, 52]
Hour:  1
Patient Arrival Time: [4, 5, 6, 18, 22, 27, 31, 36, 37, 40, 45, 46, 48, 52]
Hour:  2
Patient Arrival Time: [2, 11, 25, 32, 40, 50, 59]
Hour:  3
Patient Arrival Time: [0, 1, 8, 10, 27, 36, 57, 57]
Hour:  4
Patient Arrival Time: [1, 12, 17, 20, 21, 22, 23, 27, 36, 38, 55]
Hour:  5
Patient Arrival Time: [12, 12, 15, 18, 45, 46, 51]
Hour:  6
Patient Arrival Time: [6, 8, 10, 11, 16, 20, 21, 50]
Hour:  7
Patient Arrival Time: [14, 17, 26, 47, 58]
Hour:  8
Patient Arrival Time: [7, 8, 16, 20, 25, 27, 29, 56]
Hour:  9
Patient Arrival Time: [0, 15, 17, 18, 26, 32, 37, 43, 46, 46, 48, 53, 59]


,patient_id,arrival_time,acuity_level,treatment_plan_arr,treatment_totaltime_arr
0,1,14,5,"[1, 2, 3, 5, 4, 9]","[1, 1, 16, 1, 18, 1]"
1,2,15,4,"[1, 2, 3, 7, 5, 6, 9]","[1, 1, 12, 24, 1, 11, 1]"
2,3,19,3,"[1, 2, 3, 4, 8]","[1, 1, 15, 44, 30]"
3,4,26,4,"[1, 2, 3, 7, 8]","[1, 1, 9, 19, 30]"
4,5,31,2,"[1, 2, 3, 6, 8]","[1, 1, 14, 2, 30]"
5,6,35,4,"[1, 2, 3, 5, 4, 9]","[1, 1, 27, 1, 55, 1]"
6,7,43,1,"[1, 2, 3, 7, 8]","[1, 1, 22, 22, 30]"
7,8,47,4,"[1, 2, 3, 8]","[1, 1, 11, 30]"
8,9,52,1,"[1, 2, 3, 5, 4, 8]","[1, 1, 7, 1, 45, 30]"
9,10,64,5,"[1, 2, 3, 6, 8]","[1, 1, 12, 38, 30]"
